In [11]:
# Retrieval Augmented Generation using Azure documentation
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI


config = dotenv_values(".env")

llm = ChatOpenAI(api_key=config["OPEN_AI_KEY"])

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://learn.microsoft.com/en-us/rest/api/azure/")

docs = loader.load()

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_KEY"])

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "HOW DO I REGISTER MY CLIENT APPLICATION WITH MICROSOFT ENTRA ID?"})
print(response["answer"])


To register your client application with Microsoft Entra ID, you need to follow these steps:

1. Make sure you have a Microsoft Entra tenant. If you don't have one, you can set up a Microsoft Entra tenant.

2. Understand the two types of clients supported by Microsoft Entra ID: web/confidential clients and native/public clients.

3. Register your client application with Microsoft Entra ID by creating an application object and a service principal object in the Microsoft Entra tenant where the application is registered.

4. Set permission requests to allow the client to access the Azure Resource Manager API and configure Azure Resource Manager Role-Based Access Control (RBAC) settings for authorizing the client.

5. If your client accesses an API other than an Azure Resource Manager API, refer to the specific documentation for registering applications with the Microsoft identity platform.

6. Consider creating a secret key if you are registering a web client.

7. Configure your applicati